In [9]:
import xarray as xr
import dask

#Questions:
# level refers to pressure?
# 100m wind inferred from 10m wind?
# solar power inferred from 'mean_surface_net_short_wsave_radiation_flux',

#ds = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2023_01_10-full_37-1h-0p25deg-chunk-1.zarr')
#(561264, 721, 1440, 37) 30s
#ds = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr')
#(time: 93544, lat: 721, long: 1440, level:13) 12.1s
ds = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2023_01_10-1h-240x121_equiangular_with_poles_conservative.zarr')
#(561264, 240, 121, 13) 16s
ds

<xarray.Dataset> Size: 669TB
Dimensions:                                           (time: 561264,
                                                       latitude: 721,
                                                       longitude: 1440,
                                                       level: 37)
Coordinates:
  * latitude                                          (latitude) float32 3kB ...
  * level                                             (level) int64 296B 1 .....
  * longitude                                         (longitude) float32 6kB ...
  * time                                              (time) datetime64[ns] 4MB ...
Data variables: (12/48)
    10m_u_component_of_wind                           (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_v_component_of_wind                           (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    2m_dewpoint_temperature                           (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    2m_temperature                                    (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    anisotropy_of_sub_gridscale_orography             (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    ...                                                ...
    v_component_of_wind                               (time, level, latitude, longitude) float32 86TB dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    vertical_velocity                                 (time, level, latitude, longitude) float32 86TB dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_1                     (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_2                     (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_3                     (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_4                     (time, latitude, longitude) float32 2TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>

In [ ]:
#list(ds.coords.time.values)
#list(ds.coords['time'].values)

In [10]:
list(ds.keys())

['10m_u_component_of_wind',
 '10m_v_component_of_wind',
 '2m_dewpoint_temperature',
 '2m_temperature',
 'angle_of_sub_gridscale_orography',
 'anisotropy_of_sub_gridscale_orography',
 'boundary_layer_height',
 'geopotential',
 'geopotential_at_surface',
 'high_vegetation_cover',
 'lake_cover',
 'land_sea_mask',
 'leaf_area_index_high_vegetation',
 'leaf_area_index_low_vegetation',
 'low_vegetation_cover',
 'mean_sea_level_pressure',
 'mean_surface_latent_heat_flux',
 'mean_surface_net_long_wave_radiation_flux',
 'mean_surface_net_short_wave_radiation_flux',
 'mean_surface_sensible_heat_flux',
 'mean_top_downward_short_wave_radiation_flux',
 'mean_top_net_long_wave_radiation_flux',
 'mean_top_net_short_wave_radiation_flux',
 'mean_vertically_integrated_moisture_divergence',
 'potential_vorticity',
 'sea_ice_cover',
 'sea_surface_temperature',
 'slope_of_sub_gridscale_orography',
 'snow_depth',
 'soil_type',
 'specific_humidity',
 'standard_deviation_of_filtered_subgrid_orography',
 'stan

In [11]:
lat_data = ds.latitude
# 47, 55
lat_bounds = slice(55.5, 46.5) if lat_data[0] > lat_data[-1] else slice(46.5, 55.5)

lon_data = ds.longitude
lon_bounds = slice(15, 6) if lon_data[0] > lon_data[-1] else slice(6, 15)




In [12]:
temperature_at850hpa = ds.temperature.sel(level=1000, 
                                          time='2010-01-01T00:00:00',
                                          latitude=lat_bounds,
                                          longitude=lon_bounds
                                          )
temperature_at850hpa



<xarray.DataArray 'temperature' (latitude: 37, longitude: 37)> Size: 5kB
dask.array<getitem, shape=(37, 37), dtype=float32, chunksize=(37, 37), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float32 148B 55.5 55.25 55.0 54.75 ... 47.0 46.75 46.5
    level      int64 8B 1000
  * longitude  (longitude) float32 148B 6.0 6.25 6.5 6.75 ... 14.5 14.75 15.0
    time       datetime64[ns] 8B 2010-01-01
Attributes:
    long_name:      Temperature
    short_name:     t
    standard_name:  air_temperature
    units:          K

In [13]:
# Aggregate the data into larger tiles. Here assuming that each tile is represented by a 2x2 grid.
# You need to adjust '2' based on the resolution of your dataset and the size of the new tiles you desire.
# 8,12
temperature_larger_tiles = temperature_at850hpa.coarsen(
    latitude=2, 
    longitude=2,
    boundary='trim'  # 'trim' drops any data not fitting into the window perfectly; use 'pad' to include all data.
).mean()


temperature_larger_tiles = temperature_larger_tiles.compute()

In [14]:
#df = temperature_at850hpa.to_dataframe()
df = temperature_larger_tiles.to_dataframe()
print(df.shape)
print(df)

# one time point:
# 1h resolution: 13 sec
# 6h resolution: 5 sec

# one day:
# 1h resolution: 4min
# 6h resolutioin: 15s

# one month:
# 1h
# 6h resolution: 7 min

(324, 3)
                    level       time  temperature
latitude longitude                               
55.375   6.125       1000 2010-01-01   273.956207
         6.625       1000 2010-01-01   273.253662
         7.125       1000 2010-01-01   272.617920
         7.625       1000 2010-01-01   271.846344
         8.125       1000 2010-01-01   270.909729
...                   ...        ...          ...
46.875   12.625      1000 2010-01-01   284.064392
         13.125      1000 2010-01-01   283.506653
         13.625      1000 2010-01-01   282.982544
         14.125      1000 2010-01-01   282.986145
         14.625      1000 2010-01-01   283.045776

[324 rows x 3 columns]


In [15]:
print(df.head(30))

                    level       time  temperature
latitude longitude                               
55.375   6.125       1000 2010-01-01   273.956207
         6.625       1000 2010-01-01   273.253662
         7.125       1000 2010-01-01   272.617920
         7.625       1000 2010-01-01   271.846344
         8.125       1000 2010-01-01   270.909729
         8.625       1000 2010-01-01   268.996765
         9.125       1000 2010-01-01   269.044434
         9.625       1000 2010-01-01   270.039581
         10.125      1000 2010-01-01   270.279419
         10.625      1000 2010-01-01   269.793945
         11.125      1000 2010-01-01   269.611908
         11.625      1000 2010-01-01   267.516876
         12.125      1000 2010-01-01   267.206604
         12.625      1000 2010-01-01   268.018585
         13.125      1000 2010-01-01   268.793762
         13.625      1000 2010-01-01   270.793091
         14.125      1000 2010-01-01   272.170715
         14.625      1000 2010-01-01   272.991760


In [16]:
print(df.head(20))

                    level       time  temperature
latitude longitude                               
55.375   6.125       1000 2010-01-01   273.956207
         6.625       1000 2010-01-01   273.253662
         7.125       1000 2010-01-01   272.617920
         7.625       1000 2010-01-01   271.846344
         8.125       1000 2010-01-01   270.909729
         8.625       1000 2010-01-01   268.996765
         9.125       1000 2010-01-01   269.044434
         9.625       1000 2010-01-01   270.039581
         10.125      1000 2010-01-01   270.279419
         10.625      1000 2010-01-01   269.793945
         11.125      1000 2010-01-01   269.611908
         11.625      1000 2010-01-01   267.516876
         12.125      1000 2010-01-01   267.206604
         12.625      1000 2010-01-01   268.018585
         13.125      1000 2010-01-01   268.793762
         13.625      1000 2010-01-01   270.793091
         14.125      1000 2010-01-01   272.170715
         14.625      1000 2010-01-01   272.991760


In [21]:
xarr = xr.open_dataset("./../../data/raw/CDS/adaptor.mars.internal-1714643173.784153-29261-5-fe66c237-49a3-45b4-b05f-0ac00f6d94cc.nc")
xarr


<xarray.Dataset> Size: 28MB
Dimensions:    (longitude: 37, latitude: 33, time: 720)
Coordinates:
  * longitude  (longitude) float32 148B 6.0 6.25 6.5 6.75 ... 14.5 14.75 15.0
  * latitude   (latitude) float32 132B 55.0 54.75 54.5 54.25 ... 47.5 47.25 47.0
  * time       (time) datetime64[ns] 6kB 2023-11-01 ... 2023-11-30T23:00:00
Data variables:
    u100       (time, latitude, longitude) float64 7MB ...
    v100       (time, latitude, longitude) float64 7MB ...
    t2m        (time, latitude, longitude) float64 7MB ...
    ssr        (time, latitude, longitude) float64 7MB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-02 09:46:18 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [22]:
df = xarr.to_dataframe()
df

u100      v100         t2m  ssr
longitude latitude time                                                    
6.0       55.0     2023-11-01 00:00:00 -5.527089  4.226429  283.180884  0.0
                   2023-11-01 01:00:00 -5.890768  5.648964  283.417728  0.0
                   2023-11-01 02:00:00 -7.074062  6.921149  283.784194  0.0
                   2023-11-01 03:00:00 -8.318192  8.052382  283.977296  0.0
                   2023-11-01 04:00:00 -8.897137  9.090369  284.350698  0.0
...                                          ...       ...         ...  ...
15.0      47.0     2023-11-30 19:00:00  2.937808  0.786469  273.043036  0.0
                   2023-11-30 20:00:00  3.268729  0.623832  273.160391  0.0
                   2023-11-30 21:00:00  3.241319  0.569619  273.338023  0.0
                   2023-11-30 22:00:00  3.375025 -0.113457  273.497519  0.0
                   2023-11-30 23:00:00  3.330234 -0.151045  273.633544  0.0

[879120 rows x 4 columns]